# Installation

With conda, you can install the required dependencies with:

```bash
conda install bs4 requests lxml
```


# Basic usage of BeautifulSoup

First, we import the `BeatifulSoup` class:

In [1]:
from bs4 import BeautifulSoup

We load the html source file from disk and pass the source the the BeautifulSoup constructor. We choose the "lxml" parser for XML documents, which is faster than the defaul parser that comes with BeautifulSoup:

In [4]:
src = open("list.html")
document = BeautifulSoup(src, 'lxml')
print(document)

<!DOCTYPE html>
<html>
<body>
<h2>An Unordered HTML List</h2>
<ul id="unordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>
<h2>An Ordered HTML List</h2>
<ol id="ordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ol>
</body>
</html>



### Finding tags by name

The document now contains the full html document. We can find the first occuring tag with a specific name with the `find` function. Let's find the first un-ordered list tag:

In [5]:
ulist = document.find("ul")

The result contains all tags contained in the matched tag:

In [6]:
ulist

<ul id="unordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>

The `find_all` function returns **all** tags that match the given tag name. We can use it to get a list of all list items:

In [7]:
items = ulist.find_all("li")
items

[<li>Coffee</li>, <li>Tea</li>, <li>Milk</li>]

Finally, we can loop over all items and extract their contant with the `get_text` function:

In [8]:
for item in items:
    print(item.get_text())

Coffee
Tea
Milk


Note that `find_all` is **recursive** by default. This means that we could call it the on the full `document` to get the items
of both the ordered and un-ordered lists:

In [10]:
document.find_all("li", recursive=False)

[]

### Finding tags by attributes

Sometimes the easiest way to find a tag is by its attribute name. In our examples, both lists have an `id` attribute that uniquely identifies the tables. We can also use the `find*` methods to search for attributes:


In [11]:
document.find(attrs={"id":"unordered_list"})

<ul id="unordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>

### Accessing attributes

The `ul` tag also contains a `style` attribute. Any bs4 tag behaves like a dictionary with attribute names as keys and attribute values as values:

In [12]:
ulist.attrs

{'id': 'unordered_list', 'style': 'color:#069'}

In [13]:
ulist["style"]

'color:#069'

# Downloading a table from Wikipedia

We aim to get a list of countries sorted by their population size:
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

First, let's import the required modules:

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import dateutil

This time, we load the html source directly from a website using the requests module:

In [4]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")

The web server returns a status code to indicate if the request was (un-)succesfully.
We use that status-code to check if the page was succesfully loaded:

In [5]:
assert result.status_code==200  

Next, we extract the html source and initiated BeautifulSoup:

In [6]:
src = result.content
document = BeautifulSoup(src, 'lxml')

by looking at the document, we can see that we are interested in first table. So we use `find`:

In [7]:
table = document.find("table")
table

<table class="wikitable sortable plainrowheaders" style="text-align:right">
<tbody><tr>
<th data-sort-type="number">Rank</th>
<th>Country<br/>(or dependent territory)</th>
<th>Population</th>
<th>% of world</th>
<th>Date</th>
<th class="unsortable">Source<br/>(official or UN)
</th></tr>
<tr>
<th>1
</th>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></span> <a href="/wiki/Demographics_of_China" title="Demogra

If you are not familiar with html table, read this example first: https://www.w3schools.com/html/tryit.asp?filename=tryhtml_table_intro

At this point, it is a good idea to programatically check that the table contains the correct header:

In [8]:
assert table.find("th").get_text() == "Rank"

In [11]:
rows = table.find_all("tr")  # Note: this works because find_all is resursive by default
rows

[<tr>
 <th data-sort-type="number">Rank</th>
 <th>Country<br/>(or dependent territory)</th>
 <th>Population</th>
 <th>% of world</th>
 <th>Date</th>
 <th class="unsortable">Source<br/>(official or UN)
 </th></tr>,
 <tr>
 <th>1
 </th>
 <td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></span> <a href="/wiki/Demographics_of_China" title="Demographics of China">China</a><sup class="reference" id="cite_ref-4"><a hre

In [20]:
for row in rows[1:-1]:
    cells = row.find_all(["td", "th"])
    
    cells_text = [cell.get_text(strip=True) for cell in cells]
    
    (rank, country, population, 
     percentage, updated_at, source) = cells_text
    
    # Clean population using a regular expression str("1,404,890,600") -> int(1404890600)
    population = int(re.sub(',', '', population))   
    
    # Clean country name
    country = re.findall(r'[\w\s()\.]+', country)[0]
    
    # Convert percentage to floats
    percentage = float(re.findall(r'[\d\.]+', percentage)[0])    
    
    # Convert updated_at to date object
    updated_at = dateutil.parser.parse(updated_at)    
    
    print(f"{rank}, {country}, {population}, {percentage}, {updated_at}, {source}")
    

1, China, 1404905120, 18.0, 2020-10-17 00:00:00, National population clock[3]
2, India, 1368508290, 17.5, 2020-10-17 00:00:00, National population clock[4]
3, United States, 330488021, 4.23, 2020-10-17 00:00:00, National population clock[5]
4, Indonesia, 269603400, 3.45, 2020-07-01 00:00:00, National annual projection[6]
5, Pakistan, 220892331, 2.82, 2020-07-01 00:00:00, UN Projection[2]
6, Brazil, 212210487, 2.71, 2020-10-17 00:00:00, National population clock[7]
7, Nigeria, 206139587, 2.64, 2020-07-01 00:00:00, UN Projection[2]
8, Bangladesh, 169485868, 2.17, 2020-10-17 00:00:00, National population clock[8]
9, Russia, 146748590, 1.88, 2020-01-01 00:00:00, National annual estimate[9]
10, Mexico, 127792286, 1.63, 2020-07-01 00:00:00, National annual projection[10]
11, Japan, 125810000, 1.61, 2020-09-01 00:00:00, Monthly national estimate[11]
12, Philippines, 109306068, 1.4, 2020-10-17 00:00:00, National population clock[12]
13, DR Congo, 101935800, 1.3, 2020-07-01 00:00:00, National a

**Attention**: Beautiful Soup does not execute Javascript. This means that you the code in the Google Chrome inspector might look different to the original source code. 